In [87]:
import pandas as pd
from utils.func import remove_outliers

In [88]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [89]:
def price_per_square_meter_per_postal_code(df):
    df["Price/m2"] = df["Price"] / df["SurfaceOfGood"]
    return df

In [90]:
test = test.convert_dtypes()
train = train.convert_dtypes()



In [91]:
from pandas import CategoricalDtype

kit_cat= CategoricalDtype(categories=['not installed', 'usa uninstalled', 'semi equipped', 'usa semi equipped', 'hyper equipped', 'usa hyper equipped', 'installed', 'usa installed'], ordered=True)
train["Kitchen"] = train["Kitchen"].fillna("not installed")
train["Kitchen"] = train["Kitchen"].astype(kit_cat).cat.codes

building_state_type = pd.CategoricalDtype(
    categories=[
        "not known",
        "to be done up",
        "to restore",
        "to renovate",
        "just renovated",
        "good",
        "as new",
    ],
    ordered=True,
)
train["StateOfBuilding"] = train["StateOfBuilding"].fillna("not known")
train["StateOfBuilding"] = (
    train["StateOfBuilding"].astype(building_state_type).cat.codes
)

heating_cat= CategoricalDtype(categories=['not known', 'fueloil', 'gas','carbon', 'wood', 'pellet', 'electric',  'solar'], ordered=True)
train["Heating"] = train["Heating"].fillna("not known")
train["Heating"] = (
    train["Heating"].astype(heating_cat).cat.codes
)
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 35863 entries, 95172 to 57397
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Url                35863 non-null  string 
 1   Openfire           35863 non-null  boolean
 2   Furnished          35863 non-null  boolean
 3   Terrace            35863 non-null  boolean
 4   Garden             35863 non-null  boolean
 5   PropertyId         35863 non-null  Int64  
 6   TypeOfProperty     35863 non-null  Int64  
 7   PostalCode         35863 non-null  Int64  
 8   SubtypeOfProperty  35863 non-null  string 
 9   TypeOfSale         35863 non-null  Int64  
 10  Price              35863 non-null  Int64  
 11  Kitchen            35863 non-null  int8   
 12  StateOfBuilding    35863 non-null  int8   
 13  Heating            35863 non-null  int8   
 14  Bedrooms           25645 non-null  Int64  
 15  SurfaceOfGood      23754 non-null  Int64  
 16  SwimmingPool       3586

In [92]:
train = remove_outliers(train, 'Price', 0.05, 0.95)
test = remove_outliers(test, 'Price', 0.05, 0.95)

train = price_per_square_meter_per_postal_code(train)
test = price_per_square_meter_per_postal_code(test)

In [93]:
train['SurfaceOfGood'] = train['SurfaceOfGood'].fillna(train['LivingArea'])
train = train.drop(columns=[
    'Url', 
    'SubtypeOfProperty',
    'PropertyId', 
    # 'Kitchen', 
    # 'StateOfBuilding', 
    # 'Heating',
    'Price/m2', 
    'GardenArea', 
    'NumberOfFacades', 
    'ConstructionYear',
    'LivingArea'
])

In [94]:
train = train.dropna()
train.to_parquet('train.parquet')
test.to_parquet('test.parquet')